In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd '/content/gdrive/MyDrive/DEPLOYED_MODEL'

/content/gdrive/MyDrive/DEPLOYED_MODEL


In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Flask-0.12.2-py2.py3-none-any.whl (83 kB)
  Attempting uninstall: flask
    Found existing installation: Flask 2.2.3
    Uninstalling Flask-2.2.3:
      Successfully uninstalled Flask-2.2.3


In [ ]:
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
!./ngrok authtoken "2KlbX6GgwNZXNswrJWxKeEOUiTO_6eMgUUDW7bsayk923riv1"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Flask-2.2.3-py3-none-any.whl (101 kB)
  Attempting uninstall: flask
    Found existing installation: Flask 0.12.2
    Uninstalling Flask-0.12.2:
      Successfully uninstalled Flask-0.12.2


In [ ]:
from flask import Flask, render_template, request,redirect, url_for,session
from flask_ngrok import run_with_ngrok
import cv2
from keras.models import load_model
import numpy as np
import base64
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector,Concatenate
from keras.models import Sequential, Model
from keras.utils import np_utils
from keras.preprocessing import image, sequence
import cv2
from keras_preprocessing.sequence import pad_sequences
from tqdm import tqdm


vocab = np.load('vocab.npy', allow_pickle=True)

vocab = vocab.item()

inv_vocab = {v:k for k,v in vocab.items()}


print("+"*50)
print("vocabulary loaded")


embedding_size = 128
vocab_size = len(vocab)
#vocab_size = 4108
max_len = 40


image_model = Sequential()

image_model.add(Dense(embedding_size, input_shape=(2048,), activation='relu'))
image_model.add(RepeatVector(max_len))


language_model = Sequential()

language_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))


conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs = out)

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

model.load_weights('mine_model_weights.h5')

print("="*150)
print("MODEL LOADED")

resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')


#resnet = load_model('model.h5')

print("="*150)
print("RESNET MODEL LOADED")




app = Flask(__name__)
run_with_ngrok(app) 

app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1

app.secret_key = 'sadshfklsdhlgvd'

@app.route('/')
def index():
    return render_template('home1.html')
@app.route('/f')
def index1():
    base_url = url_for('index1', _external=True)
    return render_template('camera.html', base_url=base_url)
    #return render_template('camera.html')
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    global model, resnet, vocab, inv_vocab
    # Get the image data from the request
    image_data = request.json['image_data']

    # Convert the data URI to OpenCV format
    encoded_data = image_data.split(',')[1]
    nparr = np.frombuffer(base64.decodebytes(encoded_data.encode('utf-8')), np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (224,224))

    image = np.reshape(image, (1,224,224,3))

    
    
    incept = resnet.predict(image).reshape(1,2048)

    print("="*50)
    print("Predict Features")


    text_in = ['startofseq']

    final = ''

    print("="*50)
    print("GETING Captions")

    count = 0
    while tqdm(count < 20):

        count += 1

        encoded = []
        for i in text_in:
            encoded.append(vocab[i])

        padded = pad_sequences([encoded], maxlen=max_len, padding='post', truncating='post').reshape(1,max_len)

        sampled_index = np.argmax(model.predict([incept, padded]))

        sampled_word = inv_vocab[sampled_index]

        if sampled_word != 'endofseq':
            final = final + ' ' + sampled_word

        text_in.append(sampled_word)

    print(final)
    session['caption'] = final
    return redirect(url_for('output'))
@app.route('/f/output')
def output():
      caption = session.get('caption')
      #caption = session['caption']
    # Do something with the caption
      return render_template('result2.html', data=caption)


@app.route('/result', methods=['GET', 'POST'])
def result():

    global model, resnet, vocab, inv_vocab

    img = request.files['file1']

    img.save('static/file.jpg')

    print("="*50)
    print("IMAGE SAVED")
    image = cv2.imread('static/file.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image = cv2.resize(image, (224,224))

    image = np.reshape(image, (1,224,224,3))

    
    
    incept = resnet.predict(image).reshape(1,2048)

    print("="*50)
    print("Predict Features")


    text_in = ['startofseq']

    final = ''

    print("="*50)
    print("GETING Captions")

    count = 0
    while tqdm(count < 20):

        count += 1

        encoded = []
        for i in text_in:
            encoded.append(vocab[i])

        padded = pad_sequences([encoded], maxlen=max_len, padding='post', truncating='post').reshape(1,max_len)

        sampled_index = np.argmax(model.predict([incept, padded]))

        sampled_word = inv_vocab[sampled_index]

        if sampled_word != 'endofseq':
            final = final + ' ' + sampled_word

        text_in.append(sampled_word)



    return render_template('result.html', data=final)
if __name__ == "__main__":
    app.run()

++++++++++++++++++++++++++++++++++++++++++++++++++
vocabulary loaded
MODEL LOADED
RESNET MODEL LOADED
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1da5-34-125-248-140.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [22/Apr/2023 08:59:24] "GET /f HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2023 08:59:25] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 1s/step
Predict Features
GETING Captions


...

0it [00:00, ?it/s]


1/1 [==============================] - 0s 70ms/step


0it [00:00, ?it/s]
INFO:werkzeug:127.0.0.1 - - [22/Apr/2023 09:03:31] "POST /upload HTTP/1.1" 302 -


 a person jumps off a grassy trail . . . .


INFO:werkzeug:127.0.0.1 - - [22/Apr/2023 09:03:32] "GET /f/output HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Apr/2023 09:03:36] "GET /f/output HTTP/1.1" 200 -
